In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementNotInteractableException
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
from tqdm import tqdm

# SETTING UP DRIVER

In [10]:
def driver():
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    s = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    #options.add_argument("headless")
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument("--window-size=1920,1080")
    #options.binary_location = '/Applications/Brave Browser.app/Contents/MacOS/Brave Browser'
    return webdriver.Chrome(service=s, options=options)
    
def linkedin_login():
    mail = 'cocazapata.21@gmail.com'
    pw = 'fcbarcelona21'
    driver.get('https://www.linkedin.com')
    username = driver.find_element(By.ID, 'email-or-phone').send_keys(mail)
    password = driver.find_element(By.ID, 'password')
    password.send_keys(pw)
    password.send_keys(Keys.ENTER)

# SCRAPPING LINKEDIN 

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import random
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import numpy as np

def data_analyst():
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    s = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    # options.add_argument("headless")
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument("--window-size=1920,1080")
    options.binary_location = '/Applications/Brave Browser.app/Contents/MacOS/Brave Browser'
    driver = webdriver.Chrome(service=s, options=options)
    mail = 'cocazapata.21@gmail.com'
    pw = 'fcbarcelona21'
    driver.get('https://www.linkedin.com')
    username = driver.find_element(By.ID, 'session_key').send_keys(mail)
    password = driver.find_element(By.ID, 'session_password').send_keys(pw)
    #password.send_keys(Keys.ENTER)

    time.sleep(20)

    list_of_dictionaries = []

    for i in tqdm(range(0, 976, 25)):
        #driver.get(f'''https://www.linkedin.com/jobs/search/?f_TPR=r2592000&geoId=107025191&keywords=data&location=Barcelona%2C%20Catalonia%2C%20Spain&start={i}''')
        driver.get(f'''https://www.linkedin.com/jobs/search/?currentJobId=3652284026&geoId=107025191&keywords=data&location=Barcelona%2C%20Catalonia%2C%20Spain&start={i}''')
        time.sleep(random.randint(3, 5))
        all_listings = []
        keep_scrolling = True

        while keep_scrolling:
            listings = driver.find_elements(By.CSS_SELECTOR, ".job-card-list__title")
            if set(listings) == set(all_listings):
                keep_scrolling = False
            else:
                new_listings = list(set(listings) - set(all_listings))
                for listing in new_listings:
                    all_listings.append(listing)
                    try:
                        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth'});", listing)
                        time.sleep(random.randint(1, 3))
                        listing.click()
                        time.sleep(random.randint(2, 5))
                        page_source = driver.page_source
                        soup = BeautifulSoup(page_source, "html.parser")
                        try:
                            posicioncita  = {}
                            title = driver.find_element(By.XPATH, "//*[starts-with(@id, 'ember')]/h2").text
                            posicioncita["title"] = title
                            
                            if 'data' in title.lower() or 'datos' in title.lower():
                                
                                try:
                                    company = driver.find_element(By.XPATH, "//*[contains(@class, 'ember-view t-black t-normal')]").text
                                    posicioncita["companies"] = company
                                except:
                                    posicioncita["companies"] = np.nan
                                try:
                                    location = driver.find_element(By.XPATH, "//*[contains(@class, 'jobs-unified-top-card__bullet')]").text
                                    posicioncita["location"] = location
                                except:
                                    posicioncita["location"] = np.nan
                                try:
                                    modality = soup.select('span.jobs-unified-top-card__workplace-type')[0].get_text()
                                    posicioncita["modality"] = modality
                                except:
                                    posicioncita["modality"] = np.nan
                                try:
                                    description = soup.select('#job-details > span')[0].get_text()
                                    posicioncita["description"] = description
                                except:
                                    posicioncita["description"] = np.nan
                            
                            list_of_dictionaries.append(posicioncita)
                                    
                        except NoSuchElementException:
                            driver.save_screenshot(f"{random.randint(0, 100)}.png")
                    except StaleElementReferenceException:
                        driver.save_screenshot(f"{random.randint(0, 100)}.png")
                time.sleep(random.randint(1, 2))
    driver.quit()
    df = pd.DataFrame(list_of_dictionaries)
    return df

In [13]:
analyst = data_analyst()

100%|████████████████████████████████████████| 40/40 [1:36:51<00:00, 145.29s/it]


In [46]:
analyst

,title,companies,location,modality,description
0,Data Analyst,ID Finance,"Barcelona, Catalonia, Spain",On-site,"\nROLE PURPOSETo collect, organize and interpr..."
1,Data Specialist,Loyal Guru,"Barcelona, Catalonia, Spain",Hybrid,\nLoyal Guru is the leading Customer Data Plat...
2,Data Analyst,The Knot Worldwide,"Sant Cugat del Vallès, Catalonia, Spain",NaN,\nJob Description\nWHAT WE DO MATTERS:\nHere a...
3,BI Analyst,NaN,NaN,NaN,NaN
5,Data Scientist,ClearPeaks,"Barcelona, Catalonia, Spain",Hybrid,\nAre you passionate about Business Intelligen...
...,...,...,...,...,...
803,Senior Platform Engineer - Data As A Service,eDreams ODIGEO,"Barcelona, Catalonia, Spain",Hybrid,\n We are looking for a seasone...
806,Senior Azure Data Engineer,Gartner,"Barcelona, Catalonia, Spain",Hybrid,\nAbout Gartner IT: \nJoin a world-class team ...
814,"Senior Scientist, Computational Oncology, Onco...",AstraZeneca,"Barcelona, Catalonia, Spain",NaN,\n The Computational Oncology g...
847,"Associate Director, Computational Oncology, On...",AstraZeneca,"Barcelona, Catalonia, Spain",NaN,\n The Computational Oncology g...


In [15]:
analyst.drop_duplicates(subset=['description'], inplace=True)


In [16]:
analyst.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 319 entries, 0 to 860
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        319 non-null    object
 1   companies    317 non-null    object
 2   location     318 non-null    object
 3   modality     272 non-null    object
 4   description  318 non-null    object
dtypes: object(5)
memory usage: 15.0+ KB


In [48]:
analyst.rename(columns={'companies': 'company'}, inplace=True)


# CONCATING DIFFERENT DF FROM DIFFERENT SCRAPPING TIMES

In [71]:
# Deleting all rows that contains more than 2 NaN values:
nan_counts = combined_df.isna().sum(axis=1)

filtered_df = analyst[nan_counts < 2]
print(filtered_df)

                                                 title             company  \
0                                         Data Analyst          ID Finance   
1                                      Data Specialist          Loyal Guru   
2                                         Data Analyst  The Knot Worldwide   
3                                           BI Analyst                 NaN   
5                                       Data Scientist          ClearPeaks   
..                                                 ...                 ...   
803       Senior Platform Engineer - Data As A Service      eDreams ODIGEO   
806                         Senior Azure Data Engineer             Gartner   
814  Senior Scientist, Computational Oncology, Onco...         AstraZeneca   
847  Associate Director, Computational Oncology, On...         AstraZeneca   
860  Senior Data Engineer – Apache Spark (Java/Scal...               Agoda   

                                    location modality  \
0     

/var/folders/q7/v3v0dp3x75b1z141v2hlcr3c0000gn/T/ipykernel_53699/1316820303.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df = analyst[nan_counts < 2]


In [76]:
df2 = pd.read_csv('DATA_JOBS.csv')

In [77]:
df1 = pd.read_csv('DATA_JOBS_BCN.csv')

In [78]:
df_final = pd.concat([df1, df2], ignore_index=True)


In [79]:
df_final.drop_duplicates(subset=['description'], inplace=True)


In [80]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 424 entries, 0 to 694
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        424 non-null    object
 1   company      422 non-null    object
 2   location     423 non-null    object
 3   modality     406 non-null    object
 4   description  423 non-null    object
 5   companies    0 non-null      object
dtypes: object(6)
memory usage: 23.2+ KB


In [82]:
df_final.to_csv("DATA_JOBS_JULIO.csv", index=False)
